<a href="https://colab.research.google.com/github/Paoarlett10/Tarea2/blob/main/ejercicio21_b).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 21.b)
Vazquez Jimenez Arlett Paola / 320162033


**Factorización LU**


In [1]:

import numpy as np
from numpy import linalg as LA

In [23]:
#Inciso b)
import numpy as np
A=np.array([[1.,4.,-2.,1.],[-2.,-4.,-3.,1.],[1.,16.,-17.,9.],[2.,4.,-9.,-3.]])
b=np.array([[3.5],[-2.5],[15.],[10.5]])
print(A)
print(b)

[[  1.   4.  -2.   1.]
 [ -2.  -4.  -3.   1.]
 [  1.  16. -17.   9.]
 [  2.   4.  -9.  -3.]]
[[ 3.5]
 [-2.5]
 [15. ]
 [10.5]]


In [9]:
def SustitucionDelante(Mat, b):
    n = Mat.shape[0]
    x = np.zeros(n)

    for i in range(n):
        SumCum = 0.0
        for j in range(i):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]

    return x

In [10]:

def SustitucionAtras(Mat, b):
    n = Mat.shape[0]
    x = np.zeros(n)

    for i in range(n-1, -1, -1):
        SumCum = 0.0
        for j in range(i+1, n):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]

    return x


In [11]:
def LU(A):
  U=np.copy(A)
  L=np.eye(A.shape[0])

  for i in range(A.shape[0]):
    Li=np.eye(A.shape[0])
    for j in range(i+1,A.shape[0]):
      Li[j,i]=-U[j,i]/U[i,i]
      L[j,i]=U[j,i]/U[i,i]
    U=Li@U
  return L,U

In [12]:

def SolverLU(A,b):
  L,U=LU(A)
  # El sistema que se resuelve es Ly=b
  y=SustitucionDelante(L, b)
  # El sistema que se resuelve es Ux=y
  x=SustitucionAtras(U, y)

  return x

In [13]:
Sol=SolverLU(A,b)
Sol

<ipython-input-9-6e7392762ea7>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i] = (b[i] - SumCum) / Mat[i, i]


array([-0.5,  1. , -0.5, -1. ])

In [14]:
LA.solve(A,b)

array([[-0.5],
       [ 1. ],
       [-0.5],
       [-1. ]])

**Factorización LU con pivoteo parcial**


In [15]:
import numpy as np
from numpy import linalg as LA
A


array([[  1.,   4.,  -2.,   1.],
       [ -2.,  -4.,  -3.,   1.],
       [  1.,  16., -17.,   9.],
       [  2.,   4.,  -9.,  -3.]])

In [17]:

A[[0,3]]=A[[3,0]]
A

array([[  2.,   4.,  -9.,  -3.],
       [ -2.,  -4.,  -3.,   1.],
       [  1.,  16., -17.,   9.],
       [  1.,   4.,  -2.,   1.]])

In [18]:
j=1
k=np.argmax(np.abs(A[j:,j]))+j
k

np.int64(2)

In [19]:

def Permutaciones(A,b):
  U=np.copy(A)
  x=np.copy(b)
  P=np.eye(A.shape[0])

  for j in range(A.shape[0]):
    k=np.argmax(np.abs(A[j:,j]))+j
    U[[j,k]]=U[[k,j]]
    P[[j,k]]=P[[k,j]]
    x[[j,k]]=x[[k,j]]

  return P,U,x

In [20]:
P,U,x=Permutaciones(A,b)
U

array([[  2.,   4.,  -9.,  -3.],
       [  1.,  16., -17.,   9.],
       [ -2.,  -4.,  -3.,   1.],
       [  1.,   4.,  -2.,   1.]])

In [21]:
def Solver_LU_Pivot_Partial(A,b):
    P,Ap,bp=Permutaciones(A,b)
    x=SolverLU(Ap,bp)

    return x


In [22]:
SOl=Solver_LU_Pivot_Partial(A,b)
print("La solucion es: ",SOl)
print("La solucion analítica es: ",LA.solve(A,b))

La solucion es:  [ 63.95833333 -21.82291667  -4.29166667  25.25      ]
La solucion analítica es:  [[ 63.95833333]
 [-21.82291667]
 [ -4.29166667]
 [ 25.25      ]]


<ipython-input-9-6e7392762ea7>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i] = (b[i] - SumCum) / Mat[i, i]


**Factorización LU con pivoteo total**

In [24]:
def Permutaciones(A, b):
    n = A.shape[0]  # Tamaño de la matriz (n x n)
    U = np.copy(A)  # Copia de la matriz A para no modificarla directamente
    x = np.copy(b)  # Copia del vector b
    P = np.eye(n)   # Matriz de permutación de filas (inicialmente identidad)
    Q = np.eye(n)   # Matriz de permutación de columnas (inicialmente identidad)

    for k in range(n - 1):
        # --- PASO 1: Encontrar el pivote máximo en la submatriz U[k:, k:] ---
        # np.abs(U[k:, k:]): Valores absolutos de la submatriz desde (k,k) hasta el final
        # np.argmax(...): Índice lineal del elemento con mayor valor absoluto (en arreglo aplanado)
        # np.unravel_index(...): Convierte el índice lineal a coordenadas (fila, columna) en la submatriz
        max_row, max_col = np.unravel_index(np.argmax(np.abs(U[k:, k:])), (n - k, n - k))

        # Ajustar índices para referenciar la posición en la matriz original U (no solo la submatriz)
        max_row += k
        max_col += k

        # --- PASO 2: Intercambiar filas para llevar el pivote a la posición (k,k) ---
        # Intercambia filas en U
        U[[k, max_row]] = U[[max_row, k]]
        # Intercambia filas en la matriz de permutación P
        P[[k, max_row]] = P[[max_row, k]]
        # Intercambia elementos en el vector x
        x[[k, max_row]] = x[[max_row, k]]

        # --- PASO 3: Intercambiar columnas para optimizar la factorización ---
        # Intercambia columnas en U
        U[:, [k, max_col]] = U[:, [max_col, k]]
        # Intercambia columnas en la matriz de permutación Q
        Q[:, [k, max_col]] = Q[:, [max_col, k]]

    return P, Q, U, x  # Devuelve: Matrices de permutación, matriz triangular, y vector modificado

In [25]:
P,Q,U,x=Permutaciones(A,b)
U

array([[-17.,  16.,   9.,   1.],
       [ -3.,  -4.,   1.,  -2.],
       [ -9.,   4.,  -3.,   2.],
       [ -2.,   4.,   1.,   1.]])

In [26]:
def Solver_LU_Pivot_Total(A,b):
    P,Q,A_g,b_g=Permutaciones(A,b)
    x=SolverLU(A_g,b_g)
    x=Q@x

    return x

In [28]:
SOl=Solver_LU_Pivot_Total(A,b)
print("La solucion es: ",SOl)
print("La solucion analítica es: ",LA.solve(A,b))

La solucion es:  [-0.5  1.  -0.5 -1. ]
La solucion analítica es:  [[-0.5]
 [ 1. ]
 [-0.5]
 [-1. ]]


<ipython-input-9-6e7392762ea7>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i] = (b[i] - SumCum) / Mat[i, i]
